In [2]:
import csv
import json
import numpy as np
import math
import xlrd
import matplotlib.pyplot as plt

In [91]:
'''
reading raw data from tsv file into raw_data
'''
keys = {'timestamp': 0, 'SearchQuery': 1, 'SearchKeyword': 2, 'finalAssetIdList': 3, 'myfinalAssetIdList': 4}
def read_data(file_path):
    count = 0
    error = 0
    raw_data = []
    
    with open(file_path, encoding='utf-16') as tsvfile:
        tsvreader = csv.reader(tsvfile, delimiter="\t")
        skip = next(tsvreader)
        while True:
            try:
                row = next(tsvreader)
                if len(row) != 4:
                    error += 1
                    continue

                dictionary = row[1]
                dictionary = json.loads(dictionary)
                raw_data.append([row[0], row[2], row[3], dictionary])
                count += 1
            except UnicodeDecodeError:
                error += 1
                continue
            except StopIteration:
                break
    print("number of all data = {}".format(count))
    print("number of all errors occured(either unicode error or rows with irregular number of items) = {}".format(error))
    return raw_data

raw_data = read_data("dt29to3/dt29to3.tsv")
# raw_data = read_data("data.tsv")

number of all data = 1141606
number of all errors occured(either unicode error or rows with irregular number of items) = 6407


In [92]:
'''
reading the real cvr data from the file and saving them into cvr which is a dictionary with keys (advertiser_id, keyword)
'''
loc = ("Maleki_AssetDetails.xlsx") 
  
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0) 


cvr = {}
impression = {}
for i in range(1, sheet.nrows):
    cvr[(int(sheet.cell_value(i, 0)),sheet.cell_value(i, 1))] = sheet.cell_value(i, 5)
    impression[(int(sheet.cell_value(i, 0)),sheet.cell_value(i, 1))] = int(sheet.cell_value(i, 2))
#     if sheet.cell_value(i, 5) != 0:
#         cvr[(int(sheet.cell_value(i, 0)),sheet.cell_value(i, 1))] = sheet.cell_value(i, 5)
        
kws = set()
ads = set()
for i in range(1, sheet.nrows):
    ads.add(int(sheet.cell_value(i, 0)))
    kws.add(sheet.cell_value(i, 1))
print("total number of pairs having true cvrs is {}".format(len(cvr.keys())))
print("number of distinct keywords = {} and number of distinct advertisers in true cvr data = {}".format(len(list(kws)), len(list(ads))))
# print(cvr)
# print(type(sheet.cell_value(3,0)))

total number of pairs having true cvrs is 5875
number of distinct keywords = 3942 and number of distinct advertisers in true cvr data = 57


In [93]:
loc = ("Asset_keyword_price.xlsx") 
  
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(0)

adv_kw = []

for i in range(1, sheet.nrows):
    if type(sheet.cell_value(i, 2)) == float:
        temp = (int(sheet.cell_value(i, 0)), str(sheet.cell_value(i, 1)), sheet.cell_value(i, 2))
    adv_kw.append(temp)
print("total number of {} advertiser-keyword pairs were found".format(len(adv_kw)))

total number of 28565 advertiser-keyword pairs were found


In [95]:
count1, count2 = 0, 0
for (adv_id, kw, bid) in adv_kw:
    if (adv_id, kw) in cvr.keys():
        count1 += 1
    count2 += 1
print("In {} pairs, {} of them had true cvr.".format(count2, count1))

In 28565 pairs, 5857 of them had true cvr.


In [96]:
def match_keyword(search_query, keywords):
    keyword = ""
    for word in search_query:
        if word in keywords:
            keyword += word + " "
    return keyword[:-1]

def get_all_bidders(query):
    all_assets = []
    search_query = query[keys['SearchQuery']].split()
    
    for (adv_id, kw, bid) in adv_kw:
        kw = kw.split()
        flag = True
        for keyword in kw:
            if keyword not in search_query:
                flag = False
        
        if flag:
            all_assets += [(adv_id, kw, bid)]
    all_assets_dict = {}
    for asset in all_assets:
        all_assets_dict[asset[0]] = []
    for asset in all_assets:
        all_assets_dict[asset[0]].append((asset[1], asset[2]))
    
    final_asset_list = []
    for asset_id in all_assets_dict.keys():
        bigram = False
        bid = 0
        keywords_combined = []
        for item in all_assets_dict[asset_id]:
            bid = max(bid, item[1])
            keywords_combined += item[0]
        if len(keywords_combined) > 1:
            bigram = True
        keywords_combined = list(set(keywords_combined))
        matched_keyword = match_keyword(search_query, keywords_combined)
        temp_dict = {'Id': asset_id, 'Kw': matched_keyword, 'Pr':bid, 'Bigram':bigram}
        final_asset_list.append(temp_dict)
    return final_asset_list

i = 3
# print(raw_data[i])
print(raw_data[i][keys['SearchQuery']])
# print(raw_data[i][keys['finalAssetIdList']])
get_all_bidders(raw_data[i])            

واتساپ


[{'Id': 14239, 'Kw': 'واتساپ', 'Pr': 1500.0, 'Bigram': False},
 {'Id': 14243, 'Kw': 'واتساپ', 'Pr': 2000.0, 'Bigram': False},
 {'Id': 14241, 'Kw': 'واتساپ', 'Pr': 1800.0, 'Bigram': False},
 {'Id': 14316, 'Kw': 'واتساپ', 'Pr': 1500.0, 'Bigram': False},
 {'Id': 14423, 'Kw': 'واتساپ', 'Pr': 1500.0, 'Bigram': False}]

In [97]:
for i in range(len(adv_kw)):
    if type(adv_kw[i][2]) != float:
        print(i , adv_kw[i][2])

In [98]:
for i in range(len(raw_data)):
    raw_data[i].append(get_all_bidders(raw_data[i]))
    if i % 100000 == 0:
        print('{} queries has been processed.'.format(i))


0 queries has been processed.


KeyboardInterrupt: 

In [9]:
'''
We should remove those data from the true cvr which we do not have any test data. First we 
'''
all_assets = []
for row in raw_data:
    all_assets += row[keys['finalAssetIdList']]
print("numebr of all assets combined together = {}".format(len(all_assets)))
count = 0
for row in raw_data:
    if len(row[keys['finalAssetIdList']]) == 1:
        count += 1
print("number of all queries with only one asset = {}".format(count))


all_pairs = set()
for asset in all_assets:
    all_pairs.add((asset['Id'], asset['Kw']))

count = 0
c = 0
cvr_keys = list(cvr.keys())
for key in cvr_keys:
    if key not in all_pairs:
        del cvr[key]
        count += 1
    c += 1
print("number of all pairs having true cvr = {} and number of pairs in cvr that are not in any asset in data = {}".format(c, count))
print("Therefore, we have {} pairs of true cvr.".format(len(cvr.keys())))

numebr of all assets combined together = 1848066
number of all queries with only one asset = 693563
number of all pairs having true cvr = 5875 and number of pairs in cvr that are not in any asset in data = 1845
Therefore, we have 4030 pairs of true cvr.


In [26]:
raw_data_del = []
count1, count2 = 0, 0
for row in raw_data:
    assets = row[keys['finalAssetIdList']]
    for asset in assets:
        if len(asset['Kw'].split()) > 1:
            count1 += 1
        count2 += 1

print("In total {} number of assets, {} of them were matched to a bigram or more.".format(count2, count1))
        
count1, count2 = 0, 0
for i in range(len(raw_data)):
    assets = raw_data[i][keys['finalAssetIdList']]
    flag1, flag2 = False, False
    temp_asset = []
    for asset in assets:
        if len(asset['Kw'].split()) > 1:
            flag1 = True
            temp_asset.append(asset)
        else:
            flag2 = True
    
    raw_data_del.append(raw_data[i])
    
    if flag1 and flag2:
        count1 += 1
        raw_data_del[i][keys['finalAssetIdList']] = temp_asset
#         raw_data[i].append(True)
#     else:
#         raw_data[i].append(False)
    count2 += 1

print("In total {} number of queries, {} of them had both bigram and unigram assets.".format(count2, count1))

In total 1765147 number of assets, 286427 of them were matched to a bigram or more.
In total 1141606 number of queries, 28355 of them had both bigram and unigram assets.


In [31]:
len(raw_data_del)

1141606

In [32]:
'''
removing those assets whose true cvr is not available. if a query instance has no asset with true cvr data, then the
whole query will be removed.
'''
data = []
for row in raw_data:
    temp = []
    for bidder in row[keys['finalAssetIdList']]:
        if (bidder['Id'], bidder['Kw']) in cvr.keys():
            temp.append(bidder)
    row[keys['finalAssetIdList']] = temp
    if len(temp) > 0:
        data.append(row)

print("number of queries after filtering = {}".format(len(data)))

number of queries after filtering = 1100804


In [33]:
'''
removing those assets whose true cvr is not available. if a query instance has no asset with true cvr data, then the
whole query will be removed.
'''
data_del = []
for row in raw_data_del:
    temp = []
    for bidder in row[keys['finalAssetIdList']]:
        if (bidder['Id'], bidder['Kw']) in cvr.keys():
            temp.append(bidder)
    row[keys['finalAssetIdList']] = temp
    if len(temp) > 0:
        data_del.append(row)

print("number of queries after filtering = {}".format(len(data_del)))

number of queries after filtering = 1100804


In [34]:
'''
We should keep the true cvr dataset and give it to both models as prior information. We keep the true cvr data with 
probability = ratio
'''
def initialize_priors(ratio=0.8):
    prior_cvr = {}
    prior_n_ij = {}
    for key in cvr.keys():
        if np.random.binomial(1, ratio):
            prior_cvr[key] = cvr[key]
            prior_n_ij[key] = impression[key]
    
    return prior_cvr, prior_n_ij
prior_cvr, prior_n_ij = initialize_priors()
print("{} of {} cvrs were selected as prior information.".format(len(prior_cvr.keys()), len(cvr.keys())))

3192 of 4030 cvrs were selected as prior information.


In [35]:
'''
initializing the model parameters. cvr_hat, n_ij and n_j
'''
def get_parameters(reinit=False, ratio=0.8):
    if reinit:
        p_cvr, p_n_ij = initialize_priors(ratio)
    else:
        p_cvr, p_n_ij = prior_cvr, prior_n_ij
    
    cvr_hat = {}
    n_ij = {}
    n_j = {}
    for key in cvr.keys():
        cvr_hat[key] = p_cvr.get(key, 0)
        n_ij[key] = p_n_ij.get(key, 0)
        n_j[key[1]] = 0
    return cvr_hat, n_ij, n_j

In [ ]:
def determine_bidders(query, cvr_hat):
    all_bidders = query[keys['finalAssetIdList']]
    have_cvr = []
    for bidder in all_bidders:
        if cvr_hat[(bidder['Id'], bidder['Kw'])] != 0:
            have_cvr.append(True)
        else:
            have_cvr.append(False)
    
    m = have_cvr.count(True)
    n = len(have_cvr)
    exploit = np.random.binomial(1, m/n)
    if exploit == 0:
        return all_bidders, exploit
    
    bidders_with_cvr = []
    for i in range(len(all_bidders)):
        if have_cvr[i]:
            bidders_with_cvr.append(all_bidders[i])
    
    return bidders_with_cvr, exploit

def old_allocation(bidders, exploit, cvr_hat):
    score = [bidder['Pr'] for bidder in bidders]
    if exploit == 1:
        for i in range(len(bidders)):
            score[i] *= cvr_hat[(bidders[i]['Id'], bidders[i]['Kw'])]
    
    sorted_idx = list((-np.array(score)).argsort())
    if len(sorted_idx) == 1:
        sorted_idx.append(None)
    return {'winner': sorted_idx[0], 'second': sorted_idx[1]}

def old_payment(bidders, exploit, winner, second, cvr_hat):
    if second is None:
        if bidders[winner]['Pr'] > 3000:
            return bidders[winner]['Pr'] / 2
        return max(1500, bidders[winner]['Pr'])
    
    if exploit == 0:
        payment = bidders[second]['Pr']
        if payment >= 1500:
            return payment
        return max(1500, bidders[winner]['Pr'] / 2)
    
    c1 = cvr_hat[(bidders[winner]['Id'], bidders[winner]['Kw'])]
    c2 = cvr_hat[(bidders[second]['Id'], bidders[second]['Kw'])]
    payment = (bidders[second]['Pr'] * c2) / c1
    if payment >= 1500:
        return payment
    return max(1500, bidders[winner]['Pr'] / 2)


def old_parse_query(query, cvr_hat, n_ij):
    bidders, exploit = determine_bidders(query, cvr_hat)
    
    allocation = old_allocation(bidders, exploit, cvr_hat)
    winner, second = allocation['winner'], allocation['second']
    
    revenue = old_payment(bidders, exploit, winner, second, cvr_hat)
    if revenue < 1500:
        raise ValueError("reveune is {}".format(revenue))
        
#     vase inke payment bishtar az bid e taraf nashe!
#     if query[keys['finalAssetIdList']][winner]['Pr'] < revenue:
#         print(query)
#     revenue = min(revenue, query[keys['finalAssetIdList']][winner]['Pr'])
    
    advertiser_id = bidders[winner]['Id']
    keyword = bidders[winner]['Kw']
    result = is_installed(cvr[(advertiser_id, keyword)])
    
    total_revenue = revenue * result
    
    cvr_hat[(advertiser_id, keyword)] = ((cvr_hat[(advertiser_id, keyword)] * n_ij[(advertiser_id, keyword)]) + result) / (n_ij[(advertiser_id, keyword)] + 1)
    n_ij[(advertiser_id, keyword)] += 1
    
    return total_revenue

def old_run(data, n=1, reinit=True, ratio=0.8):
    old_total_revenue = 0
    for _ in range(n):
        cvr_hat, n_ij, _ = get_parameters(reinit=reinit, ratio=ratio)
        
        for row in data:
            old_total_revenue += old_parse_query(row, cvr_hat, n_ij)
            
    
    for i in range(len(data)):
        old_total_revenue[i] /= n
    
    return old_total_revenue
#     old_total_revenue = 0
#     for i in range(n):
#         cvr_hat, n_ij, _ = get_parameters(reinit=reinit, ratio=ratio)
        
#         for row in data:
#             old_total_revenue += old_parse_query(row, cvr_hat, n_ij)
#     return old_total_revenue / n